In [13]:
!pip install gradio
!pip install scikit-learn==1.5.2
import pandas as pd
import numpy as np
import gradio as gr
import joblib
from tensorflow.keras.models import load_model


# === Load models ===
catboost_model = joblib.load('/kaggle/input/checkpoint/tensorflow2/default/1/catboost_model.pkl')
lasso_model = joblib.load('/kaggle/input/checkpoint/tensorflow2/default/1/lasso_model.pkl')
random_forest_model = joblib.load('/kaggle/input/checkpoint/tensorflow2/default/1/lasso_model.pkl')
ridge_model = joblib.load('/kaggle/input/checkpoint/tensorflow2/default/1/ridge_best_model.pkl')
xgboost_model = joblib.load('/kaggle/input/checkpoint/tensorflow2/default/1/xgboost_model.pkl')
linear_regression_model = joblib.load('/kaggle/input/checkpoint/tensorflow2/default/1/linear_regression_model.pkl')
neural_network_model = load_model('/kaggle/input/checkpoint/tensorflow2/default/1/nn_model.keras')
meta_model_xgb=joblib.load('/kaggle/input/checkpoint/tensorflow2/default/1/meta_model_xgb.pkl')
meta_model_catboost=joblib.load('/kaggle/input/checkpoint/tensorflow2/default/1/meta_model_catboost.pkl')
scaler = joblib.load('/kaggle/input/standard/tensorflow2/default/1/scaler_standard.pkl')

# === Map functions ===
def map_club(file_path):
    df = pd.read_csv(file_path)
    return dict(zip(df['player_club'], df['rank']))

def map_nationality(file_path):
    df = pd.read_csv(file_path)
    return dict(zip(df['nationality'], df['rank']))

club_dict = map_club("/kaggle/input/ranking/Clubs-ranking-copy.csv")
nation_dict = map_nationality("/kaggle/input/ranking/fifa_ranking.csv")

# === Preprocessing ===
def sample_x(player_club, age, nationality, player_height, contract_value_time, appearances, PPG, substitutions_on,
             substitutions_off, goals_conceded, clean_sheet, minute_per_game, goal_per_game, assist_per_game,
             position_Defender, position_Forward, position_Goalkeeper, position_Midfielder, strong_foot_both,
             strong_foot_left, strong_foot_right):
    club_rank = club_dict.get(player_club, 0)
    nation_rank = nation_dict.get(nationality, 0)

    x = [
        club_rank, age, nation_rank, player_height, contract_value_time, appearances, PPG, substitutions_on,
        substitutions_off, goals_conceded, clean_sheet, minute_per_game, goal_per_game, assist_per_game,
        position_Defender, position_Forward, position_Goalkeeper, position_Midfielder, strong_foot_both,
        strong_foot_left, strong_foot_right
    ]
    return np.array(x, dtype=float)

# === Prediction functions ===
def predict_with_models(X):
    predictions = {
        'catboost': catboost_model.predict(X),
        'lasso': lasso_model.predict(X),
        'random_forest': random_forest_model.predict(X),
        'ridge': ridge_model.predict(X),
        'xgboost': xgboost_model.predict(X),
        'linear_regression': linear_regression_model.predict(X),
        'neural_network': neural_network_model.predict(X).flatten()
    }
    return predictions

def ensemble_predict(X, meta_model):
    base_predictions = predict_with_models(X)
    base_df = pd.DataFrame(base_predictions)

    best_model_indices = meta_model.predict(base_df)
    final_predictions = [base_df.iloc[i, idx] for i, idx in enumerate(best_model_indices)]
    return np.array(final_predictions)

# === Gradio backend ===
def predict(player_club, age, nationality, player_height, contract_value_time, appearances, PPG, substitutions_on,
            substitutions_off, goals_conceded, clean_sheet, minute_per_game, goal_per_game, assist_per_game,
            position, strong_foot, model_choice):
    
    # Convert position and strong_foot to boolean flags
    position_Defender = 1 if position.lower() == 'defender' else 0
    position_Forward = 1 if position.lower() == 'forward' else 0
    position_Goalkeeper = 1 if position.lower() == 'goalkeeper' else 0
    position_Midfielder = 1 if position.lower() == 'midfielder' else 0
    strong_foot_both = 1 if strong_foot.lower() == 'both' else 0
    strong_foot_left = 1 if strong_foot.lower() == 'left' else 0
    strong_foot_right = 1 if strong_foot.lower() == 'right' else 0

    # Create test sample
    test_sample = sample_x(
        player_club, age, nationality, player_height, contract_value_time, appearances, PPG,
        substitutions_on, substitutions_off, goals_conceded, clean_sheet, minute_per_game,
        goal_per_game, assist_per_game, position_Defender, position_Forward, position_Goalkeeper,
        position_Midfielder, strong_foot_both, strong_foot_left, strong_foot_right
    )
    test_sample = test_sample.reshape(1, -1)

    # Scale features
    features_to_normalize = test_sample[:, :14]
    remaining_features = test_sample[:, 14:]
    features_to_normalize_scaled = scaler.transform(features_to_normalize)
    test_sample_scaled = np.hstack([features_to_normalize_scaled, remaining_features])

    # Predict based on model choice
    if "catboost" in model_choice.lower() and "ensemble" not in model_choice.lower():
        return catboost_model.predict(test_sample_scaled)[0]
    elif "lasso" in model_choice.lower():
        return lasso_model.predict(test_sample_scaled)[0]
    elif "random_forest" in model_choice.lower():
        return random_forest_model.predict(test_sample_scaled)[0]
    elif "ridge" in model_choice.lower():
        return ridge_model.predict(test_sample_scaled)[0]
    elif "xgboost" in model_choice.lower() and "ensemble" not in model_choice.lower():
        return xgboost_model.predict(test_sample_scaled)[0]
    elif "linear_regression" in model_choice.lower():
        return linear_regression_model.predict(test_sample_scaled)[0]
    elif "neural_network" in model_choice.lower():
        return neural_network_model.predict(test_sample_scaled)[0][0]
    elif "ensemble" in model_choice.lower():
        if "catboost" in model_choice.lower():
            return ensemble_predict(test_sample_scaled, meta_model_catboost)[0]
        else:
            return ensemble_predict(test_sample_scaled, meta_model_xgb)[0]
    else:
        return "Invalid model choice!"

# === Gradio UI ===


/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.6.0 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [14]:
import gradio as gr

css_code = """
.gradio-container {
    min-width: var(--size-full) !important;
    background-image: url('https://images.pexels.com/photos/46798/the-ball-stadion-football-the-pitch-46798.jpeg');
    background-size: cover;
    background-repeat: no-repeat;
    background-attachment: fixed;
    background-position: center;
    animation: backgroundFade 15s infinite alternate;
}

@keyframes backgroundFade {
    0% {
        filter: brightness(0.8);
    }
    50% {
        filter: brightness(1);
    }
    100% {
        filter: brightness(0.8);
    }
}

/* Custom class for transparent input rows */
.transparent-row {
    background: rgba(255, 255, 255, 0.3); /* Semi-transparent white */
    padding: 10px;
    border-radius: 10px;
    box-shadow: 0px 4px 6px rgba(0, 0, 0, 0.1);
}

.transparent-input {
    background: rgba(255, 255, 255, 0.3); /* Semi-transparent background */
    padding: 10px;
    border-radius: 10px;
    border: 1px solid rgba(255, 255, 255, 0.6); /* Subtle border */
    color: #000; /* Text color */
    font-size: 16px;
    box-shadow: 0px 4px 6px rgba(0, 0, 0, 0.1); /* Optional: Box shadow */
}

.transparent-input input::placeholder {
    color: rgba(0, 0, 0, 0.5); /* Placeholder text color */
}
"""

with gr.Blocks(css=css_code) as demo:
    gr.Markdown("### Football Player Market Value Prediction")
    with gr.Row(elem_classes='transparent-row'):
        club = gr.Textbox(label="Player Club", elem_classes='transparent-input')
        age = gr.Number(label="Age", elem_classes='transparent-input')
        nationality = gr.Textbox(label="Nationality", elem_classes='transparent-input')
        player_height = gr.Number(label="Player Height (cm)", elem_classes='transparent-input')
        contract_value_time = gr.Number(label="Remaining Contract Time (Years)", elem_classes='transparent-input')
        appearances = gr.Number(label="Appearances", elem_classes='transparent-input')
        PPG = gr.Number(label="PPG (Points Per Game)", elem_classes='transparent-input')
    with gr.Row(elem_classes='transparent-row'):
        substitutions_on = gr.Number(label="Substitutions On", elem_classes='transparent-input')
        substitutions_off = gr.Number(label="Substitutions Off", elem_classes='transparent-input')
    with gr.Row(elem_classes='transparent-row'):
        goals_conceded = gr.Number(label="Goals Conceded", elem_classes='transparent-input')
        clean_sheet = gr.Number(label="Clean Sheets", elem_classes='transparent-input')
        minute_per_game = gr.Number(label="Minutes Per Game", elem_classes='transparent-input')
        goal_per_game = gr.Number(label="Goals Per Game", elem_classes='transparent-input')
        assist_per_game = gr.Number(label="Assists Per Game", elem_classes='transparent-input')
        position = gr.Textbox(label="Position (e.g., Defender, Forward, etc.)", elem_classes='transparent-input')
        strong_foot = gr.Textbox(label="Strong Foot (e.g., Left, Right, Both)", elem_classes='transparent-input')
    with gr.Row(elem_classes='transparent-row'):
        model_choice = gr.Textbox(label="Model Choice (e.g., CatBoost, Ensemble, etc.)", elem_classes='transparent-input')
    result = gr.Textbox(label="Predicted Market Value", elem_classes='transparent-input')
    submit_button = gr.Button("Predict")

    submit_button.click(
        predict,
        inputs=[
            club, age, nationality, player_height, contract_value_time, appearances, PPG, substitutions_on,
            substitutions_off, goals_conceded, clean_sheet, minute_per_game, goal_per_game, assist_per_game,
            position, strong_foot, model_choice
        ],
        outputs=result
    )

# Pass the custom CSS to the app
demo.launch(share=True, inline=False)


* Running on local URL:  http://127.0.0.1:7863
* Running on public URL: https://442e7ebd0ac1c6e8b6.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but Lasso was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but Lasso was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but Ridge was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but Lasso was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but Lasso was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but Ridge was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
